In [1]:
cd D:\Saarbrucken\EDA_Research\thesis_code\disentanglement_lib_pl

D:\Saarbrucken\EDA_Research\thesis_code\disentanglement_lib_pl


In [2]:
import torch
from torch import nn
import numpy as np
from architectures import encoders, decoders

In [3]:
enc = getattr(encoders, 'SimpleGaussianConv64CommAss')(10,3,64)
dec = getattr(decoders, 'SimpleConv64CommAss')(10,3,64)

In [4]:
img = torch.rand(size=[1,3,64,64])

In [5]:
enc

SimpleGaussianConv64CommAss(
  (main): Sequential(
    (0): Conv2d(3, 32, kernel_size=(4, 4), stride=(2, 2))
    (1): ReLU(inplace=True)
    (2): Conv2d(32, 32, kernel_size=(4, 4), stride=(2, 2))
    (3): ReLU(inplace=True)
    (4): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2))
    (5): ReLU(inplace=True)
    (6): Conv2d(64, 64, kernel_size=(4, 4), stride=(2, 2))
    (7): ReLU(inplace=True)
    (8): Flatten3D()
    (9): Linear(in_features=256, out_features=256, bias=True)
    (10): ReLU(inplace=True)
    (11): Linear(in_features=256, out_features=20, bias=True)
  )
)

In [6]:
m,s = enc(img)

torch.Size([1, 64, 2, 2])


In [7]:
m.size(), s.size()

(torch.Size([1, 10]), torch.Size([1, 10]))

In [8]:
I=dec(m)

In [9]:
I.size()

torch.Size([1, 3, 66, 66])

In [10]:
dec

SimpleConv64CommAss(
  (main): Sequential(
    (0): Unsqueeze3D()
    (1): Conv2d(10, 256, kernel_size=(1, 1), stride=(1, 1))
    (2): ReLU(inplace=True)
    (3): Conv2d(256, 1024, kernel_size=(1, 1), stride=(1, 1))
    (4): ReLU(inplace=True)
    (5): Reshape()
    (6): ConvTranspose2d(64, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): ConvTranspose2d(64, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): ConvTranspose2d(32, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): ConvTranspose2d(32, 3, kernel_size=(4, 4), stride=(2, 2))
  )
)

In [11]:
o=nn.Conv2d(10, 256, 1, 2)(torch.rand(1,10,1,1))
print(o.size())

torch.Size([1, 256, 1, 1])


In [12]:
o=nn.Conv2d(256, 64 * 4 * 4, 1, 1)(torch.rand(1, 256, 1, 1))
print(o.size())

torch.Size([1, 1024, 1, 1])


In [13]:
from common.ops import Unsqueeze3D, Reshape
r=Reshape([64, 4, 4])(o)
print(r.size())

torch.Size([1, 64, 4, 4])


In [18]:
s=nn.ConvTranspose2d(64, 64, 4, 2, 1)(r)
s.size()

torch.Size([1, 64, 8, 8])

In [19]:
t=nn.ConvTranspose2d(64, 32, 4, 2, 1)(s)
t.size()

torch.Size([1, 32, 16, 16])

In [20]:
u=nn.ConvTranspose2d(32, 32, 4, 2, 1)(t)
u.size()

torch.Size([1, 32, 32, 32])

In [23]:
v=nn.ConvTranspose2d(32, 3, 4, 2,1)(u)
v.size()

torch.Size([1, 3, 64, 64])